In [14]:
import pandas as pd
import json
import requests
from warcio import ArchiveIterator
import os 
import numpy as np 
from multiprocessing import set_start_method
from multiprocessing import get_context
from multiprocessing import pool 
import multiprocessing
from requests.adapters import HTTPAdapter
from requests.packages import urllib3
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm 
import pickle 
import re

In [12]:
trial2 = pd.read_pickle('trial_28_02.pkl')

# START HAVING A LOOK AT GETTING POSTCODES

In [22]:
def postcode_finder(text):
    postcodes = re.findall(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
    return postcodes 

In [26]:
def lambda_postcode(df):
    df['postcode'] = df.apply(lambda x: postcode_finder(x['text']), axis = 1)
    return df 

In [28]:
text_postcode = lambda_postcode(trial2)

In [34]:
dataframe = text_postcode[['url', 'url_host_name', 'warc_filename', 'warc_record_length', 'text', 'postcode']]

In [35]:
dataframe

,url,url_host_name,warc_filename,warc_record_length,text,postcode
50,https://0-0.co.uk/viewforum.php?f=6&sid=af0e73...,0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882573...,4475,Forge 1.8.9 Mods - 00\n00\nRandomness\nSkip to...,[]
51,https://0-0.co.uk/viewforum.php?f=6&sid=990559...,0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882573...,4473,Forge 1.8.9 Mods - 00\n00\nRandomness\nSkip to...,[]
52,https://0-0.co.uk/viewforum.php?f=8&sid=fda7eb...,0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882573...,5342,Microsoft - 00\n00\nRandomness\nSkip to conten...,[]
53,https://0-0.co.uk/viewforum.php?f=2,0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882573...,5713,Spigot Minecraft server - 00\n00\nRandomness\n...,[]
54,http://www.0-0.co.uk/viewtopic.php?p=17&sid=c7...,www.0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882571...,5711,Windows file and print server migration - 00\n...,[]
55,https://0-0.co.uk/viewtopic.php?p=17&sid=fda7e...,0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882573...,5779,Windows file and print server migration - 00\n...,[]
56,https://0-0.co.uk/viewtopic.php?t=6&sid=af0e73...,0-0.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882573...,5577,Install a version of Forge on Ubuntu Server - ...,[]
57,http://0-1.co.uk/,0-1.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882571...,1360,zero one : moving image design\n,[]
58,http://0-1.co.uk/ZO_alt.html,0-1.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882571...,1148,zero one : moving image design\n,[]
59,https://0-1-0test.co.uk/,0-1-0test.co.uk,crawl-data/CC-MAIN-2022-33/segments/1659882570...,8224,Day 2 & 8 PCR Test - 0-1-0test.co.uk\nHome\nRa...,[]


In [40]:
texts = dataframe[35:43]

In [42]:
texts['url'].tolist()

['https://www.001-translations.co.uk/c_132/contact.html?agencyCity=UK_cambridgeshire_cambridge_non-certified',
 'https://www.001-translations.co.uk/c_132/contact.html?agencyCity=UK_hertfordshire_watford_certified',
 'https://www.001-translations.co.uk/c_132/contact.html?agencyCity=UK_lanarkshire_manchester_certified',
 'https://www.001-translations.co.uk/c_132/contact.html?agencyCity=UK_norfolk_norwich_certified',
 'https://www.001-translations.co.uk/c_132/contact.html?agencyCity=UK_nottinghamshire_nottingham_certified',
 'https://www.001-translations.co.uk/c_132/contact.html?agencyCity=UK_west-yorkshire_bradford_certified',
 'https://www.001-translations.co.uk/c_135/careers.html?agencyCity=UK_antrim_belfast_certified',
 'https://www.001-translations.co.uk/c_135/careers.html?agencyCity=UK_bedfordshire_luton_certified']

In [41]:
texts['text'].tolist()

['\ufeff Contact - 001 Translation UK\n001 Translation UK\n📱 0800 133 7692\n001 Translation UK\nToggle navigation\nHOME\nSERVICES\nSPECIALTIES\nRATES\nOUR OFFICES\nCONTACT\nChoose your language\nEnglish | American English\nFrench | Français\nDeutsch | Allemand\nSpanish | Español\nItalian | Italiano\n1 hour Translation for Covid 19 PCR Testing\nESTIMATE IN 5 SECONDS\nLANGUAGES\nCAREERS\nRATES\nPrevious Next\n\ufeff\nNavigation\nContact\nContact us!\nTranslation Request\nInterpreting Request\nJob Opportunities\nOther Enquiry\n\ufeff\n01223 967594\ntranslate@translation-cambridge.co.uk\t\n001 Translation services - Cambridge\n35 Clifton Road\nCB1 7ED Cambridge\nUK\t\nMonday to Friday\nfrom 9 am to 6 pm\nName* Email* Please confirm your email* Phone number* Message*\n* required\nHome\nServices\nSpecialties\nRates\nLanguages\nCareers\nOur Offices\nContact\n| Choose your language\n001 Translation UK for and in collaboration with 001 Translation Inc, 8 The Green, Suite #10810, Dover, DE 19901